<H1 style="background-color:#00008b;color:#F8F8FF;"><center>
    Project Report - Phillip Marsh
    </center></H1>
<hr>

<H2 style="color:#00008b;">GitHub URL</H2>
Phillip's GutHub can be found at: <a href="https://github.com/PhillipNM/UCDPA_PhillipMarsh" target="_blank">Phillip's GitHub Repository</a>
<pre style="font-family:arial;">
document should contain between 1,500 and 2,00 words
</pre>

<h2 style="color:#00008b;">Abstract</h2>
(short overview of the entire project)

For this project I chose to review COVID data as I was somewhat familiar with the underlying data but only from creating metrics on the data.  I wanted to gain some further understanding of the situation and felt there would be a lot of data options available.  The results did not turn out as I planned but the excercise was rewarding but very challenging. Trying to cover such a large scope of skills with in python and the huge amount of imformation on tips and tricks, although many sites are not that useful and I spent hours between the DataCamp videos and onlineadvice sites.  It turns out that population density and economic prosperity of a country does not have much of an impact of a disease like COVID which I guess is why people are not panicking each flu season.  I would have loved to added some insights into the impact of masking, and lock downs but tring to join that periodic data in with this daily data was too much of a challenge for this short period of time.

<h2 style="color:#00008b;">Introduction</h2>

(Explain why you chose this project use case)

After considering serveral ideas and researching the available dataset I decide on a dataset I am fairly familiar with from a reporting point of view (as part of the business continuity team) but that I had not done much with the other than create some metrics using Tableau.  I wondered if we could predict confidently that countries with lower population densities or high GDP per capita fared better than higher density countries or lower GDP.

<h2 style="color:#00008b;">Datasets</h2>
(Provide a description of your dataset and source. Also justify why you chose this source)

<h6>Deciding on the dataset</h6>
<br>
I thought about several ideas. However, I explored three main ideas:

<pre style="font-family:arial;">
1. Predicting currency fx changes to maximise buys and sells.
As I have two children in university in Canada the fx rate for USD to CAD is always top of mind.  After exploring this for a bit the challenge to understand the market conditions that I could use for making predictions did not seem to fit well with what I needed for this project and collecting the data was to burdensome for the scope of this project.
2. Flight delays, cancellations and the average compensation.  Are the airlines "gaming" the system to not pay-out customers given the turmoil in travel I thought it would be interesting to compare recent cancellations, delays and reasons and compensations vs. pre-covid times. Researching for datasets I could not find anything current. Although, there are some pay sites which may have data.  As I would be required pay for the data I decided against this topic.
3. COVID data.  This idea would have plenty of source data out there but would it offer the ability to make predictions and not just forecasting trends. I feel this lends it's self to the machine learning section as there are so many stories about the disease so would give me an opportunity to show case what I learned.
</pre>

<h6>COVID Data</h6>
   I picked the COVID idea as there is good data and the types of calculations and techniques required would lend itself to the project with some work. This data is something we are all very familiar with at this time. Governments, countries, organizations and corporations have struggled with rules and regulations trying to balance controlling the epidemic vs. economic stability.

 I reviewed a couple of sources and in the end selected "Our World In Data" (OWID).  OWID has a comprehensive set of publicly available data specifically for COVID.  In working with the FIL business continuity team, I assited with the COVID response. I came across this data source and found it very useful. In the end this is the source we used to provide global situational updates for the senior members in the organization so they could decide on stay at home and return to office responses for each jurisdiction across the organization.  The data is quite clean so I was concerned about showing the data cleaning side of things

source of covid data: https://github.com/owid/covid-19-data/tree/master/public/data 
<p> Originally I downloaded a (.csv) copy of the data to use but the file was large (I was getting an error that the file was to big for my type of GitHub repository account). this occured when I pushed the data to my GitHub repository.  I then researched how I could link to an external csv file, and this solved the problem.  This file creats the opportunity to use current data.  However, I noticed that the most current days data is not 100% populated so I have adjusted to used the most recent data - two days.</p>

source of GDP data: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?year_high_desc=false
<p>the file is a zip file which is difficult to connect to so in this instance I downloaded the file and unzipped it.</p>


<h2 style="color:#00008b;">Implementation Process</h2>
(describe your entire process in detail)

<h6>Hypothesis</h6>
My hypothesis is that countries with higher population density and lower GDP have higher mortality rates than for higher density higher GDP countries. It would also be interesting to see how lower density and higher GDP countries fared and if density and GDP are a predictor of mortaility for a disease like COVID

<pre style="font-family:arial;">
The implementation process I followed was
    Gather Data
    Transform & clean
    Explore
    Analyze and build models
</pre
>    
<h6>Gather Data</h6>
There are several measures I need for my analysis. If any of the data sets include 0 values for total I will use the prior days data as total are cumlative.  However, the earlier dates would certainly have 0 as there may not have been cases for a country at the start date of teh data.
<pre style="font-family:arial;">
Planned measures for each country:
    Highest Cases per 100k people: for year end 2020, 2021 and latest 2022
    Highest Deaths per 100k people: for year end 2020, 2021 and latest 2022
    Lowest Cases per 100k people: for year end 2020, 2021 and latest 2022
    Lowest Deaths per 100k people: for year end 2020, 2021 and latest 2022
    Look at the 14 day rolling average cases per 100k people over time
    Look at the 14 day rolling average deaths per 100k people over time
    Population density
    GDP per person
    Mortality: high/low when cases per 100k deaths reaches a threshold to be set
</pre>
<h6>Transform & Clean and Explore</h6>
Review data for size and complexity, NaNs and missing values. Use techiques like 
<pre style="font-family:arial;">
    .head()
    .tail()
    .info()
    .shape()
    .isna().sum()
I will need to understand the number of columns, count of records and the type of objects being used: like strings, dates, intergers and floats. I will review the null records and get a count to understand the completeness of the data, and build functions to assit with exploring the data; like creating a rolling n day average and calulation for the total on a per 100,00 of the population for comparative purposes.
</pre>

<h6>Analyze and build models</h6>
Take the top 20: 
    Categorize as High, Low for mortality and add to the data set.  This will allow some of the linear regression models for correlations

Run agaisnt the machine learnng logic for insights


<h3>Import and review the data</h3>

In [1]:
# Import packages needed for project:
import pandas as pd
import requests
import io
import datetime as dt
from datetime import datetime
from datetime import timedelta
import numpy as np
from collections import Counter
import re
import sklearn

# Visualization
import matplotlib.pyplot as plt
# import matplotlib.animation as animation
import seaborn as sns

# Machine learning
#from sklearn.module import Model
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


<h5>create global variables</h5>

In [2]:
#how many columns are too many to wrangle
column_count_limit=30 #number of columns deemed to be managble for exploring data
#this will allow a use to run a calculation to high light if a detset has a large number of columns

#number of days used in rolling average default = 14 but user could change to 25,30 etc. depending on what timeframe 
#is relevant
days_calc = 14 #n days for calculations. 

top_n_parameter = 10 #variable to use for select the number of top and bottom records

pop_per_100k = 100000 #varibale to set for total cases and deaths per population

#for calculations relating to mortality
high_deaths_per_100k = 50 # was 50
low_deaths_per_100k = 10 # was 10
# I decide on this after reviewing the min and maxk values for the topn records for each year end

As I create variables used in multiple places I endavour to capture them here so I can easily make updates. This saves time when when making changes.

<h4>Gather data</h4>

In [ ]:
# Import COVID data

# Link and download COVID dataset from OWID
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv" 
download = requests.get(url).content

# Create the COVID as a pandas dataframe
covid_data_raw = pd.read_csv(io.StringIO(download.decode('utf-8')),parse_dates=['date']) #need to parse the date field
#source: https://stackoverflow.com/questions/59004960/converting-date-format-in-a-dataframe-from-a-csv-file


Review of covid header details:

In [ ]:
covid_data_raw.head()

A quick review shows there are a lot of columns of which most will be irrelivant for this work.  There are also records with NaN which will have to be dealt with as they would impact calulations.

In [ ]:
# Import World Bank GDP data
    # source: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?year_high_desc=false

    # Create the GDP raw file as a pandas dataframe, headers start on row 4
gdp_data_raw = pd.read_csv("/Users/Phillip/UDCPA_PhillipMarsh/data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4489151.csv", skiprows=4)


Review of global gdp details:

In [ ]:
gdp_data_raw.head()

A quick review shows there are also alot of columns for yeara most of which would not be relevant.  This data also uses 3 digit ISO codes which means I can use it to join to data if need be to the COVID data.

Create global calculations to be used in the analysis

There are a few calculations that will be used repeatedly and it makes sense to put them at the start of the project so they are easy to find if changes need to be made.

In [ ]:
#global calculation

# What are the range of dates in data
beg_date = min(covid_data_raw["date"]) #starting point of the available data
end_date = max(covid_data_raw["date"]) #most recent data in the file

#calculate the lastest observation form the covid data, this data is dynamic and new data comes in each day, however, 
#it can take time for new data to roll in.  This report is using the last date available less 2 days 
last_date = end_date - timedelta(2) 
last_date_n = str(last_date)

print("The COVID data starts on "+str(beg_date)+" and the most recent date is "+str(end_date))

<h5>Exploring the data</h5>

Review the headers, number of headers, type of data to undestand more about the data available

In [ ]:
# name of a dataframe with comment before and after

def name_obj(df, comment, comment2=""):
    """Create statement naming the dataframe around comment and comment2
    
    Args:
        df (dataFrame): the name of the dataframe
        comment (string): comment string which goes before the name of the dataframe
        comment2 (string): comment string which goes after the name of the dataframe (optional)
    """
    name =[x for x in globals() if globals()[x] is df][0]
    return (comment+name+comment2)

covid_data_raw_name = name_obj(covid_data_raw,"Dataframe Name is:")
gdp_data_raw_name = name_obj(gdp_data_raw,"Dataframe Name is:")

#example: test the function
print("There are two primary sourced datasets used in this project:")
print(covid_data_raw_name)
print(gdp_data_raw_name)

In [ ]:
# create functions for reviewing dataframe headers

# create a function to make list from the column header names of a dataframe
def column_headers_list(df):
    """create a list of column headers
    
    Args:
        df (DataFrame): the name of the dataframe to use
    
    Returns:
        list of column headers
    """
    
    columns_lst = df.columns.tolist() # create a list of the column headers from the dataframe
        
    return columns_lst


In [ ]:
# Count the number of items in the list from the column header names list of a dataframe

#test the function "column_headers_list"

# Raw Covid data 
columns_lst_test = column_headers_list(covid_data_raw)
columns_len_test = len(columns_lst_test)

# Test function
#print(columns_lst_test)
#print(columns_len_test)


# Raw gdp data 
columns_lst_test = column_headers_list(gdp_data_raw)
#print("There are :"+str(columns_len_test)+" header records")

# Test function
#print(columns_lst_test)
#print("There are :"+str(columns_len_test)+" header records")



I # out any testing logic here once the logic was working.

In [ ]:
# create a function determine if the data set is too wide
def columns_comment(xlist,column_count_limit=30):
    """Use column_len to decide if the dataframe is too large to manage
    
    Args:
        xlist(list): list to review
        columns_len(int): from column_headers_list function
        column_count_limit(float): limi number of columns to compare
    """
    columns_len = len(xlist)
    
    if columns_len>column_count_limit:
        comment = "There are many columns ("+str(columns_len)+"), Drop a some of them to imporve performance and the size of the file"
    else:
        comment = "Number of columns appears manageable"
    
    return comment, columns_len



From the tests I can see both datasets contain quite alot of columns with data, so I will look at each source specifically the COVID data as this source will be critical to the project

<h6>COVID Raw Data</h6>

In [ ]:
# review Covid raw data

# show the column headers and the number of columns

data = covid_data_raw

columns_len = data.shape[1] # count the number of columns in the list

covid_name = name_obj(data,"The headers from the", "Dataframe are:")
print(column_headers_list(data))

print()

description_covid_raw = name_obj(data,"The "," DataFrame has "+str(columns_len)+" columns")
#print(description_covid_raw)

print()

#print(columns_comment(column_headers_list(df))) #xlist,column_count_limit=30)

In [ ]:
#test function columns_comment()

# test for covid data
columns_lst_covid = column_headers_list(covid_data_raw) #list of headers

comment_covid = columns_comment(columns_lst_covid,)[0] #Comment string
header_len_covid = columns_comment(columns_lst_covid,column_count_limit=30)[1] #Number of items in list

print("for the COVID raw file")
print(comment_covid)

print("-"*100)

In [ ]:
# test for gdp data
columns_lst_gdp = column_headers_list(gdp_data_raw)

comment_gdp = columns_comment(columns_lst_gdp)[0]
header_len_gdp = columns_comment(columns_lst_gdp,column_count_limit=30)[1]

print("for the gdp raw file")
print(comment_gdp)

print("-"*100)

use the shape function to summarize the total number of rows and columns for each dataset:

In [ ]:
# Understanding the data 

# Information (shape) on are the records + columns

# covid raw data
print("The COVID data shape shows:")
print(covid_data_raw.shape)
print()
print("The GDP data shape shows:")
# gdp raw data
print(gdp_data_raw.shape)

There a lot of records for the COVID data, we should limit the number of days to review, but lets remove many of the columns and create a new covid_data DataFrame from the raw file

In [ ]:
# drop columns
    #source: https://datatofish.com/drop-columns-pandas-dataframe/#:~:text=Here%20is%20the%20approach%20that%20you%20can%20use,Python%20code%20to%20drop%20the%20%E2%80%98Shape%E2%80%99%20column%20is%3A
covid_data = covid_data_raw.drop([
    'continent',
    'new_cases_smoothed',
    'new_deaths_smoothed',
    'new_cases_smoothed_per_million',
    'new_deaths_smoothed_per_million',
    'icu_patients_per_million',
    'hosp_patients',
    'hosp_patients_per_million',
    'weekly_icu_admissions',
    'weekly_icu_admissions_per_million',
    'weekly_hosp_admissions',
    'weekly_hosp_admissions_per_million',
    'total_tests_per_thousand',
    'new_tests_per_thousand',
    'new_tests_smoothed',
    'tests_per_case',
    'tests_units',
    'new_vaccinations_smoothed',
    'total_vaccinations_per_hundred',
    'people_vaccinated_per_hundred',
    'people_fully_vaccinated_per_hundred',
    'total_boosters_per_hundred',
    'new_vaccinations_smoothed_per_million',
    'new_people_vaccinated_smoothed',
    'new_people_vaccinated_smoothed_per_hundred',
    'stringency_index','median_age',
    'aged_65_older',
    'aged_70_older',
    'cardiovasc_death_rate',
    'diabetes_prevalence',
    'female_smokers',
    'male_smokers',
    'handwashing_facilities',
    'hospital_beds_per_thousand',
    'life_expectancy',
    'human_development_index',
    'excess_mortality_cumulative_absolute',
    'excess_mortality_cumulative',
    'excess_mortality',
    'excess_mortality_cumulative_per_million',
    'total_cases_per_million',
    'new_cases_per_million',
    'total_deaths_per_million',
    'new_deaths_per_million',
    'reproduction_rate',
    'people_vaccinated',
    'total_boosters',
    'new_vaccinations',
    'new_tests_smoothed_per_thousand',
    'new_tests',
    'positive_rate'
    ], 
    axis=1)

covid_data.info()

Using the .info() function we now have 14 columns that look relevant to the analysis, how many specific countries are there?

I will create a function that shows unique attributes in a list

In [ ]:
# all but the first three columns are float objects; two: "iso_code", "location" are country specific categories
# create a function to get a list of unique values

# Function to get unique values
  
  
def unique(list1):
  
    # Print directly by using * symbol
    print(*Counter(list1))


In [ ]:
#list of covid countrie ISO code

# sort by ISO_code and Date
covid_data = covid_data.sort_values(['iso_code', 'location', 'date'])

#create a list of country codes from the covid data
Country_lst_covid_1 = covid_data["iso_code"].tolist()

# list the country codes    
country_iso_list = unique(Country_lst_covid_1)


Review of this object shows there are some ISO_Codes that are more than the standard 3 char length, these should be reviewed. These are related to OWID codes used for regional aggregations of country data, they can be removed.

In [ ]:
# drop the "OWID_" records

# how many records have the OWID ISO_Code?
covid_data_owid = covid_data[covid_data["iso_code"].str.contains("OWID")] #OWID data
print("OWID data shape: "+str(covid_data_owid.shape))

covid_data = covid_data[covid_data["iso_code"].str.contains("OWID")==False] #Non OWID data
print("Non OWID data shape: "+str(covid_data.shape))


We can see fewer records. I have kept a copy of the OWID aggregate data in case there is time to look at this data further.

In [ ]:
# list the country ISO Codes again
Country_lst_covid_1 = covid_data["iso_code"].tolist()

#re-run the unique records; OWID records are no longer displayed
country_ISO_list = unique(Country_lst_covid_1)

This look better the 3 digit codes line up nicely and all look correct now

In [ ]:
# show the column headers and the number of columns

df_head = covid_data

columns_len = df_head.shape[1] # count the number of columns in the list

#print(name_obj(df_head,"The headers from the ", " DataFrame are:"))
#column_headers_list(df_head)


In [ ]:
# print a summary of covid_data

# Summary of covid_data_raw file
rows = covid_data.shape[0]
cols = covid_data.shape[1]

print("The raw data file has {} rows of data".format(f"{rows:,d}"),"and {} columns".format(f"{cols:,d}"))
if cols>column_count_limit:
    print("There are many columns, drop a some of them to imporve performance and the size of the file")
else:
    print("Number of columns appears manageable")

Let mes deal with the NaN (null) data next

In [ ]:
covid_data.head(5)

In [ ]:
covid_data.tail(5)

In [ ]:
# Review Null data
# pd.set_option('display.max_rows',None)
print("Null data:")
print(covid_data.isna().sum())

Review of the columns and null data shows iso_code, loaction (country), date, population are fully populated.

population density is not populated for everything, will need to confirm that for the selected date that his is improved

total_cases, new_cases, total_deaths, new_deaths etc I would expect null data as data would not be available for all 
countries from the start of the data period, I will need to convert these to 0's

Finally I need to review "gdp_per_capita" and "population_density" data as that could impact report later on

In [ ]:
#Which fields have nan's
covid_data[covid_data.isna().any(axis=1)]


In [ ]:
# review "gdp_per_capita" and "population_density"

covid_data_gdp_pop = covid_data[["date","iso_code","location","population","population_density","gdp_per_capita"]]

covid_data_gdp_pop["population"].isnull()

#covid_data_gdp_pop[covid_data_gdp_pop.isna().any(axis=1)]

As I only plan on using certain dates, specifically year end and the most recent for the current year, dropping these NaN records should not impact the report much

In [ ]:
# drop the NaN for the population and gdp columns
covid_data_before_na = covid_data # allows me to have access to pre change data
covid_data = covid_data.dropna(subset=["population","population_density","gdp_per_capita"])
#https://www.datasciencelearner.com/pandas-dropna-remove-nan-rows-python/

In [ ]:
#review NaN data after removing records
covid_data.isna().sum()

Next I look at the value "total" columns that have NaNs that should really be 0s.  Many of these NaNs are form the earlier dates when there weren't many cases

In [ ]:
# change the NaN's to 0 for the remainder valuations
covid_data = covid_data.fillna(0)

In [ ]:
covid_data


Final data set looks good and complete.

<h6>Final review of the COVID dataset</h6>

In [ ]:
# How many records am I dealing with

#total_records = covid_data.count(axis=1)
#print(total_records)
#print("")

# show the countries/ locations in the data
print("ISO codes and Country")
print(covid_data.pivot_table(index = ["iso_code", "location"], aggfunc ="size"))

print("")
# df.size
print("Size:")
print(covid_data.size)

print("")
# df.isnull()
column_picker ="total_deaths"
covid_ttl_deaths = covid_data.filter(["iso_code", "location",column_picker])
bool_series_null =pd.isnull(covid_ttl_deaths[column_picker])

print("Null",column_picker,": ")
print(covid_ttl_deaths[bool_series_null])
#print(covid_data.isnull())

print("")
# df.notnull()
bool_series = pd.notnull(covid_ttl_deaths[column_picker])
print("Not null:")
print(covid_ttl_deaths[bool_series])

print("")
# df.describe()
print("Describe:")
print(covid_data.describe)


Looks much better there are now no nulls for the Total_deaths which will allow caculations to be performed

<h6>GDP Raw Data</h6>

In [ ]:
# review GDP data

#look at info for gdp data
print("shape of the GDP raw data:")
print(gdp_data_raw.shape)
print()


First thing we can drop many of the year columns as the covid data does not go back that far. Clean up the column headers so they are consistent with the COVID data i.e no spaces

In [ ]:
# do not need most of the columns so will remove cols 4:63
gdp_data = gdp_data_raw.drop(gdp_data_raw.iloc[:,4:63],axis = 1)

#gdp_data = gdp_data_1.drop(gdp_data_raw.iloc[:,7],axis = 1)

#convert the spaces " " to underscore "_" consitent with the COVID data
gdp_data.columns = [c.replace(' ', '_') for c in gdp_data.columns]

gdp_data.drop('Unnamed:_66', axis=1, inplace=True)

print(gdp_data.info())
print()
print(gdp_data.head())   

In [ ]:
# show the column headers and the number of columns

columns_len = gdp_data.shape[1] # count the number of columns in the list
gdp_data_name = name_obj(gdp_data,"The headers from the ", " Dataframe are:")

print(gdp_data_name)
columns_lst_gdp = column_headers_list(gdp_data)
print(columns_lst_gdp)

print()

print(name_obj(gdp_data,"The "," DataFrame has "+str(columns_len)+" columns"))

print()

#print(columns_comment())

GDP data looks much better and is ready if I need it

In [ ]:
# Correlations of the gdp_data
gdp_data.corr()

<h4>Summary of data</h4>

In [ ]:
# Summary of Covid data
print("summary of Covid data")
print()

# Number of unique countries
n = covid_data.iso_code.nunique()
print("No of unique countries (covid_data):",n)

print("")

# Number of unique dates
n = covid_data.date.nunique()



print("No of unique dates: ",n)
print("From: ",beg_date.strftime("%b %d %Y")," to: ",end_date.strftime("%b %d %Y"))
print("")

# Number of records
rec = covid_data.shape[0]
col = covid_data.shape[1]

print("No of rows: ",f"{rec:,d}")
print("No of columns: ",f"{col:,d}") 
#source: https://stackoverflow.com/questions/60934535/format-integer-with-comma-using-python-printf
print("")

In [ ]:
# Summary of GDP data

print("summary of GDP Data")
print()


# Number of unique countries
n = gdp_data.Country_Code.nunique()
print("No of unique countries: ",n)
print("")

# Number of records
rec = gdp_data.shape[0]
col = gdp_data.shape[1]

print("No of rows: ",f"{rec:,d}")
print("No of columns: ",f"{col:,d}") 

<h6>calculations for reporting</h6>

In [ ]:
# Calculations for report:

# date calculations
# There needs to be a n_day (number of days) total for certain total columns to in turn calculate 
# comparative data against 100k of a countries population


First create a rolling n_days average per 100k of teh population. I need use the record "date" and go back 14days. Except if the date is with in 14days of the start of the dataset

In [ ]:
# n day calculations can't begin until the nth day after the first date in the dataset (beg_date)
first_calc_date = beg_date +  timedelta(days=days_calc)
print("Begining Date: "+str(beg_date)+"; Earliest starting date for calculations: "+str((first_calc_date)))

# calculate the start date for the n days data for each record
n_day_start = covid_data["date"] - timedelta(days=days_calc)
print()
print("show that the dates are populating with different results")
print(n_day_start)
print("its working")
print()

# Insert a column with the n day start date, this shows when the n days rolling date can start calculating
covid_data.insert(loc=3, column="n_day_start_date", value=n_day_start, allow_duplicates=False) 
#false will not allow the column to be entered more than once


In [ ]:
print(covid_data.info())
print()
print("the new column is now appearing")

Create the n_rolling days functions and insert into the DataFrame

In [ ]:
# n days totals 
# (https://stackoverflow.com/questions/28236305/how-do-i-sum-values-in-a-column-that-match-a-given-condition-using-pandas)
# https://python.tutorialink.com/calculate-14-day-rolling-average-on-data-with-two-hierarchies/

covid_data.sort_values(['iso_code','date'], ascending=(True,True), inplace=True)

# Rolling new cases 
rolling_new_cases = covid_data.groupby(['iso_code'])['new_cases'].transform(lambda x: x.rolling(days_calc, 1).sum())

# Insert a column with the "n" rolling new cases

#new_column string name
new_column = str(days_calc)+"_days_rolling_new_cases"
print(new_column)

print(new_column in covid_data.columns) # Test for existing column# True

#  delete new column, use if re-runing with out resetting the data, 
#if time allows will create if statement to check if column is available then deleting else insert the column
#del covid_data[str(days_calc)+"_days_rolling_new_cases"]

# insert new_column
covid_data.insert(loc=6, column=str(days_calc)+"_days_rolling_new_cases", value=rolling_new_cases, allow_duplicates=False)

print("-"*100)

# Rolling new deaths 
rolling_new_deaths = covid_data.groupby(['iso_code'])['new_deaths'].transform(lambda x: x.rolling(days_calc, 1).sum())


# Insert a column with the "n" rolling new deaths

#new_column string name
new_column_2 = str(days_calc)+"_days_rolling_new_deaths"
print(new_column_2)

print(new_column_2 in covid_data.columns) # Test for existing column# True

# delete new column
#del covid_data[str(days_calc)+"_days_rolling_new_deaths"]

# insert new_column
covid_data.insert(loc=9, column=str(days_calc)+"_days_rolling_new_deaths", value=rolling_new_cases, allow_duplicates=False)

print("-"*100)


#repeat for new deaths
#still need to create calculations for: 
    #total_cases_per_100k per 100k of the population (total_cases/population * 100,000)
    #total_deaths_per_100k of the population (total_deaths/population * 100,000)
    #total_cases_per_100sqkm of the country (total_cases/total country sqkm *100,000)
    #total_deaths_per_100sqkm of the country (total_deaths/total country sqkm *100,000)
#these will be used to use machine learning to establish if the GDP or pop density had an impact on the mortality
#merge in the gdp data if required

Create the total_100k of the population so we can compare countries if need be

In [ ]:
# Cases per 100K of population 
total_cases_per_100k = covid_data.total_cases/covid_data.population * pop_per_100k

# Insert a column total cases per 100k

#new_column string name
new_column_3 = "total_cases_per_100k"
print(new_column_3)

print(new_column_3 in covid_data.columns) # Test for existing column# True

#  delete new column
#del covid_data["total_cases_per_100k"]

# insert new_column
covid_data.insert(loc=6, column="total_cases_per_100k", value=total_cases_per_100k, allow_duplicates=False)

print("-"*100)



In [ ]:
# deaths per 100K of population 
total_deaths_per_100k = covid_data.total_deaths/covid_data.population * pop_per_100k

# Insert a column total deaths per 100k

#new_column string name
new_column_4 = "total_deaths_per_100k"
print(new_column_4)

print(new_column_4 in covid_data.columns) # Test for existing column# True

#  delete new column
#del covid_data["total_deaths_per_100k"]

# insert new_column
covid_data.insert(loc=6, column="total_deaths_per_100k", value=total_deaths_per_100k, allow_duplicates=False)

print("-"*100)

For the regressions I need to create a subset of the data to look at

Take the data for year end for 2020, 2021 and the most recent data from 2022
review this data and create the top deaths and the bottom deaths sets of data.
In the top_n death set look at the minimum value to set the lower threshold for the "HigH" mortalility classificaion
In the bottom_n deaths look at the max value to set the upper threshold for the "Low" mortalility classificaion

In [ ]:
# create a classification for mortality if the total_deaths per 100k is high or low
# obesrvations of the deaths for 2020 and 2021, get the min value of the top records:
#covid_data[covid_data["date"].isin(["2020-12-31","2021-12-31","2022-09-15"])]#,last_date])]

#last_date_n = str(last_date_n = str(last_date))

# filter data for the dates:
covid_data_observe = covid_data[covid_data["date"].isin(["2020-12-31","2021-12-31",last_date_n])]
covid_data_observe_20_21 =covid_data[covid_data["date"].isin(["2020-12-31","2021-12-31"])]
covid_data_observe_22 =covid_data[covid_data["date"].isin([last_date_n])]

#top and bottom observations
top_20_21 = covid_data_observe_20_21.nlargest(n=top_n_parameter, columns=["total_deaths_per_100k"])
bot_22 = covid_data_observe_22.nsmallest(n=top_n_parameter, columns=["total_deaths_per_100k"])

# min value in the Top mortality (top deaths) data
print("min of 2020/21 top deaths/ 100k: "+str(top_20_21["total_deaths_per_100k"].min()))
print("max of 2020/21 top deaths/ 100k: "+str(top_20_21["total_deaths_per_100k"].max()))
print("these records look very high, it could be due to an outlier, I have recalculted below again once more of the data is cleaned")
print("-"*100)

# max value in the bottom mortality (bottom deaths) data
print("max of 2022 lowest deaths/ 100k: "+str(bot_22["total_deaths_per_100k"].max()))
print("min of 2022 lowest deaths/ 100k: "+str(bot_22["total_deaths_per_100k"].min()))



In [ ]:
# classifiers for deaths beig high should be above 50 per 100k and below 10 per 100k of the population
print("high_deaths = "+str(high_deaths_per_100k))
print("low_deaths = "+str(low_deaths_per_100k))
# set these variables at the top of the project to cahnge

Add the classification to the covid_data.

In [ ]:
# create a calculation to insert the classification group of "Low" (10,25,50) to "high"
#if the total_deaths_per_100k >= 50 then "High Deaths" elseif total_deaths_per_100k <= 10 then "low Deaths"

covid_data.loc[covid_data["total_deaths_per_100k"] <= low_deaths_per_100k, "mortality"] = "low" 
covid_data.loc[covid_data["total_deaths_per_100k"] >= high_deaths_per_100k, "mortality"] = "high" 

covid_data.tail()

It can be observerd from the .tail function that observations between 10 to 50 will be blank

In [ ]:
# Review covid data using the iso code for a country
covid_data_country = covid_data[covid_data["iso_code"]=="ABW"]

covid_data_country.head(20)


In [ ]:
covid_data_country.tail(20)

I observe the earlier data in the .head() function shows empty mortality data but the later .tail() "high"'s can be seen and the matches the logic expected

In [ ]:
# are there any null values introduced by this process
print(covid_data.shape)
print()
print(covid_data.isna().sum())

In [ ]:
# need to balnk out the NaN's created in the mortality description field
covid_data = covid_data.fillna("")

print(covid_data.shape)
print()

All the new columns and records are the same in the datset now. Let's review the .corr() function for some quick insights

In [ ]:
# Correlations of the covid_data
covid_data.corr()



This does not look very promising but it's hard to read. Let's review as a heat map

In [ ]:
#Correlation of population density
corr = covid_data.corr()
plt.figure(figsize=(20, 9))
k = 12 #number of variables for heatmap
cols = corr.nlargest(k, 'population_density')['population_density'].index
cm = np.corrcoef(covid_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values,cmap="Blues")
plt.title("Correlation of population density", size = 15)
plt.show()



In [ ]:
#Correlation of gdp
corr = covid_data.corr()
plt.figure(figsize=(20, 9))
k = 12 #number of variables for heatmap
cols = corr.nlargest(k, 'gdp_per_capita')['gdp_per_capita'].index
cm = np.corrcoef(covid_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values,cmap="Blues")
plt.title("Correlation of GDP per Capita", size = 15)
plt.show()

Darker shadeing represent positive correlation.  From this we can infer that population density and gdp are not correlated to the mortality rate of a country.  GDP appears to have slightly better correllation than the population density.

In [ ]:
# not sure if we need some sort of index key to view the data, I created one I can insert if I need it later
pk = covid_data["iso_code"]+str(covid_data['date'])

#print(pk.head())

#insert pk into covid_data
#del covid_data["pk"] #delete pk column
#covid_data.insert(0, 'pk', pk)


#covid_data["pk"]
#covid_data.info()

Summary of data exploration and preparation:  the data is ready for anlyis but my confidence level is not high after reviewing the .corr() results.  I decided to leave the GDP data source at this point as the COVID GDP per capita data looks to be a good representation of the data.  Next step is to pefrom regession and machine learning, although given the low correlation I am seeing not sure how fruitful it will be

<h2 style="color:#00008b;">Analysis</h2>

<h3>Basic Charts</h3>

In [ ]:
# Set chart sizes to wide

plt.rcParams['figure.figsize'] = [15, 5]

#list of iso codes
Country_review = covid_data["iso_code"].tolist()

# list the country codes    
country_iso_list = unique(Country_review)


Select a country code above to test the chart below

In [ ]:
country_use_code = "GBR"

#covid_data = covid_data.set_index("pk")

iso_use = country_use_code

print("Total Cases vs "+new_column+" for: "+iso_use)


# Basic plot of total covid cases over time
fig, ax=plt.subplots(2,1, sharey=True)


data=covid_data[covid_data["iso_code"]==iso_use]
data1=covid_data[covid_data["iso_code"]==iso_use]

ax[0].plot(data["date"], data["total_cases"], color='b')
print()
ax[1].plot(data["date"], data[new_column], color='g') #using the new_column function above that identifies the name of the new column

ax[1].set_xlabel("Date")

plt.show()

The charts above give an indication of the total mortality per 100k people and rolling 14 day spikes representing the waves over time.  we can see that the total increases sharply between the end of 2020, 2021 and is now leveling off.

<h3>analysis with seaborn</h3>

I'm going to create a subset of the data as mentioned between year end totals/100k population to compare and see if the gdp, martality calssification impacts the results

In [ ]:
# create a subset of the COVID data for use with seaborn analysis

covid_data_small = covid_data[['date',
                               'iso_code',
                               'location',
                               'total_cases',
                               'total_cases_per_100k',
                               'total_deaths',
                               'total_deaths_per_100k',
                               'population',
                               'population_density',
                               'gdp_per_capita',
                               'extreme_poverty',
                               'people_fully_vaccinated',
                               'mortality'
                              ]]


#covid_data_small.fillna(0)
covid_data_small

last_date_n = str(last_date)

print("last date to use: "+last_date_n)




In [ ]:
# filter on dates for analysis
covid_data_sns = covid_data_small[covid_data_small["date"].isin(["2020-12-31","2021-12-31",last_date_n])]

print("covid_data_sns.shape") 
print(covid_data_sns.shape) 
print("-"*100)
print()
print(covid_data_sns.head())
print("-"*100)
print(covid_data_sns.tail())
print("-"*100)
print()
print("Null data")
print(covid_data_sns.isna().sum())
print("-"*100)
print()
print(covid_data_sns.corr())
print("-"*100)

I can see that we are getting multiple dates and requested columns of data back so good to move forward. Mortality is showing high, low and blank values


In [ ]:
sns.scatterplot(x="gdp_per_capita",y="population",data=covid_data_sns)
plt.title("Scatter plot of GPD per Capita vs Population")
plt.show()

Looks like a few outliers with a few high gdp nodes with relatively low populations

In [ ]:
sns.scatterplot(x="gdp_per_capita",y="total_deaths_per_100k",data=covid_data_sns)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.title("Scatter plot of GDP per Capita vs. Mortality per 100k", size = 15)
plt.show()

looks like an intersting visual and that we can what appears to be a pattern between countries, GDP and mortality

In [ ]:
#create data sets for each year
df_2020 = covid_data_sns[covid_data_sns["date"].isin(["2020-12-31"])]
df_2021 = covid_data_sns[covid_data_sns["date"].isin(["2021-12-31"])]
df_2022 = covid_data_sns[covid_data_sns["date"].isin(["2022-09-15"])]


In [ ]:
sns.scatterplot(x="gdp_per_capita",y="total_deaths_per_100k",data=df_2022, hue="location")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.title("Scatter plot of GDP per Capita vs. Mortality per 100k for 2022", size = 15)
plt.show()

only displaying a single point in time gets rid of some noise and it would appear looking at the latest data here that there appears to be a relationship between gdp and mortality rates.  it might be easier to review by the top and bottom countries

In [ ]:
# Top n data; use: top_n_parameter 
#https://datascientyst.com/get-top-10-highest-lowest-values-pandas/
#df.nlargest; df.nsmallest


print("Top countries by cases and deaths:")
print()

df_2020 = covid_data_sns[covid_data_sns["date"].isin(["2020-12-31"])]
df_2021 = covid_data_sns[covid_data_sns["date"].isin(["2021-12-31"])]
df_2022 = covid_data_sns[covid_data_sns["date"].isin([last_date_n])]



In [ ]:
print("creating a sets of top n cases and deaths per 100k of the population")
print()
print("Bottom countries by cases and deaths:")
print()

top_df_2020_cases_per_100k = df_2020.nlargest(n=top_n_parameter, columns=["total_cases_per_100k"])
print("top_df_2020_cases_per_100k")
print(top_df_2020_cases_per_100k)
print("-"*100)

top_df_2020_deaths_per_100k = df_2020.nlargest(n=top_n_parameter, columns=["total_deaths_per_100k"])
print("top_df_2020_deaths_per_100k")
print(top_df_2020_deaths_per_100k)
print("-"*100)

top_df_2021_cases_per_100k = df_2021.nlargest(n=top_n_parameter, columns=["total_cases_per_100k"])
print("top_df_2021_cases_per_100k")
print(top_df_2021_cases_per_100k)
print("-"*100)

top_df_2021_deaths_per_100k = df_2021.nlargest(n=top_n_parameter, columns=["total_deaths_per_100k"])
print("top_df_2021_deaths_per_100k")
print(top_df_2021_deaths_per_100k)
print("-"*100)

top_df_2022_cases_per_100k = df_2022.nlargest(n=top_n_parameter, columns=["total_cases_per_100k"])
print("top_df_2022_cases_per_100k")
print(top_df_2022_cases_per_100k)
print("-"*100)

top_df_2022_deaths_per_100k = df_2022.nlargest(n=top_n_parameter, columns=["total_deaths_per_100k"])
print("top_df_2022_deaths_per_100k")
print(top_df_2022_deaths_per_100k)




In [ ]:
print("creating a sets of bottom n cases and deaths per 100k of the population")
print()

# Bottom n data; use: top_n_parameter 
#https://datascientyst.com/get-top-10-highest-lowest-values-pandas/
#df.nlargest; df.nsmallest
print("Bottom countries by cases and deaths:")
print()

bot_df_2020_cases_per_100k = df_2020.nsmallest(n=top_n_parameter, columns=["total_cases_per_100k"])
print("bot_df_2020_cases_per_100k")
print(top_df_2020_cases_per_100k)
print("-"*100)
print()

bot_df_2020_deaths_per_100k = df_2020.nsmallest(n=top_n_parameter, columns=["total_deaths_per_100k"])
print("bot_df_2020_deaths_per_100k")
print(top_df_2020_deaths_per_100k)
print("-"*100)
print()

bot_df_2021_cases_per_100k = df_2021.nsmallest(n=top_n_parameter, columns=["total_cases_per_100k"])
print("bot_df_2021_cases_per_100k")
print(top_df_2021_cases_per_100k)
print("-"*100)
print()

bot_df_2021_deaths_per_100k = df_2021.nsmallest(n=top_n_parameter, columns=["total_deaths_per_100k"])
print("bot_df_2021_deaths_per_100k")
print(top_df_2021_deaths_per_100k)
print("-"*100)
print()

bot_df_2022_cases_per_100k = df_2022.nsmallest(n=top_n_parameter, columns=["total_cases_per_100k"])
print("bot_df_2022_cases_per_100k")
print(top_df_2022_cases_per_100k)
print("-"*100)
print()

bot_df_2022_deaths_per_100k = df_2022.nsmallest(n=top_n_parameter, columns=["total_deaths_per_100k"])
print("bot_df_2022_deaths_per_100k")
print(top_df_2022_deaths_per_100k)
print("-"*100)
print()


In [ ]:

# min value in the Top mortality (top deaths) data
print("min of 2020 top deaths/ 100k: "+str(top_df_2020_deaths_per_100k["total_deaths_per_100k"].min()))
print("max of 2020 top deaths/ 100k: "+str(top_df_2020_deaths_per_100k["total_deaths_per_100k"].max()))
print("-"*100)
print("min of 2021 top deaths/ 100k: "+str(top_df_2021_deaths_per_100k["total_deaths_per_100k"].min()))
print("max of 2021 top deaths/ 100k: "+str(top_df_2021_deaths_per_100k["total_deaths_per_100k"].max()))
print("-"*100)

# max value in the bottom mortality (bottom deaths) data
print("max of 2022 lowest deaths/ 100k: "+str(bot_df_2022_deaths_per_100k["total_deaths_per_100k"].max()))
print("min of 2022 lowest deaths/ 100k: "+str(bot_df_2020_deaths_per_100k["total_deaths_per_100k"].min()))


In [ ]:
print("Top and Bottom Mortality vs GDP per capita")
print()

#Top deaths vs gdp
sns.scatterplot(x="gdp_per_capita",y="total_deaths_per_100k",data=top_df_2022_deaths_per_100k, hue="location")
plt.title("High Deaths per 100k vs GDP, 2022", size = 15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

#bottom deaths vs gdp
sns.scatterplot(x="gdp_per_capita",y="total_deaths_per_100k",data=bot_df_2022_deaths_per_100k, hue="location")
plt.title("Low Deaths per 100k vs GDP, 2022", size = 15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

this looks strange, high mortality and gdp does not look related.  it appears from the top chart that the higher mortality countries also have higher gdp, for the most part these countries look like smaller nations.  lets look by population density

In [ ]:
print("Top and Bottom Mortality vs Population Density")

#Top deaths vs population_density
sns.scatterplot(x="population_density",y="total_deaths_per_100k",data=top_df_2022_deaths_per_100k, hue="location")
plt.title("High Deaths per 100k vs Pop. density, 2022", size = 15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

#bottom deaths vs population_density
sns.scatterplot(x="population_density",y="total_deaths_per_100k",data=bot_df_2022_deaths_per_100k, hue="location")
plt.title("Low Deaths per 100k vs Pop., 2022", size = 15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

this looks strange as well, higher mortality and lower density looks negatively related, it's difficult to tell because of the outlier, Macao, in the lower chart.  it appears from the top chart that the lesser dense countries also have higer mortality, for the most part these countries look like smaller nations. 

In [ ]:
# calculate the threshold to use for the "high" and "low" mortality column

# min value in the Top mortality (top deaths) data
print("min 2020 top deaths/ 100k: "+str(top_df_2020_cases_per_100k["total_deaths_per_100k"].min()))
print("max 2020 top deaths/ 100k: "+str(top_df_2020_cases_per_100k["total_deaths_per_100k"].max()))
print("-"*100)

# max value in the bottom mortality (bottom deaths) data
print("max 2022 lowest deaths/ 100k: "+str(bot_df_2022_cases_per_100k["total_deaths_per_100k"].max()))
print("min 2022 lowest deaths/ 100k: "+str(bot_df_2022_cases_per_100k["total_deaths_per_100k"].min()))


I used this to set the group bads for the "Mortality" column.  Using the 2020 top 10 records to set the lower limit of "high" mortality and current 2022 bottom 10 records to set the higher limit for the "low" mortality

In [ ]:
#Mortality per 100k and GDP per capita

#High mortality vs gdp
sns.relplot(x="gdp_per_capita",
            y="total_deaths_per_100k",
            data=covid_data_sns, 
            kind="scatter",
            col = "date")
plt.show()

#bottom deaths vs gdp
#sns.scatterplot(x="gdp_per_capita",y="total_deaths_per_100k",data=bot_df_2020_cases_per_100k, hue="location")
#plt.show()

we can observe from this that these are expected results; over time lower gdp per capita records had higher mortality per 100k of the population. However, there are some interesting results where lower gdp did not have a high mortality.  notice the skew to the upper left over time which suggests lower gdp does impact higher mortality

In [ ]:
#Mortality per 100k and population density

#High mortality vs pop density
sns.relplot(x="population_density",
            y="total_deaths_per_100k",
            data=covid_data_sns, 
            kind="scatter",
            col = "date")
plt.show()

#bottom deaths vs gdp
#sns.scatterplot(x="gdp_per_capita",y="total_deaths_per_100k",data=bot_df_2020_cases_per_100k, hue="location")
#plt.show()

we can observe here that a higher density in the population does not have a more significant impact on mortality

In [ ]:
covid_temp = covid_data_sns
covid_temp.sort_values("population_density", axis = 0, ascending = False, inplace = True, na_position ='last')
covid_temp

<h3>Regression analysis</h3>

In [ ]:
# Machine Learning


In [ ]:
# Machine learning KNN data

In [ ]:
covid_data_sns.info()


In [ ]:
date_list = unique(covid_data_sns["date"])
print("we have the expected 3 dates selected")

In [ ]:
#create 2 sets of data for the review removing date, iso code, location, as this was causing an issue with the regression, 
#and even fewer value columns in the second set 

covid_data_ml = covid_data_sns

covid_data_ml1 = covid_data_ml.drop(["date","iso_code","location"],axis=1)

covid_data_ml2 = covid_data_ml.drop(["date","iso_code","location","total_cases", "total_deaths"],axis=1)

covid_data_ml2

In [ ]:
covid_data_ml2.corr()

In [ ]:
#heat map of the correlations for covid_data_ml2

Corr_data = covid_data_ml2

#Correlation of gdp
corr = Corr_data.corr()
plt.figure(figsize=(20, 9))

k = 12 #number of variables for heatmap

cols = corr.nlargest(k, 'gdp_per_capita')['gdp_per_capita'].index
cm = np.corrcoef(Corr_data[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values,cmap="Blues")
plt.title("heat map of the correlations for covid_data_ml")
plt.show()

<h2 style="color:#00008b;">Results</h2>
(Include the charts and describe them)

<h4>Supervised learning with classification</h4>

In [ ]:
print("training and testing the data")
#from datacamp
print()

#covid_data_ml
#covid_data_ml1
#covid_data_ml2 

ml_data = covid_data_ml1

X = ml_data.drop("mortality",axis=1).values #drop target value
y = ml_data["mortality"].values #target observations

#split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=21,stratify=y)
knn = KNeighborsClassifier(n_neighbors=3)

#fit the classiier to the training data
knn.fit(X_train, y_train)

#print the accuracy
print("The knn score:")
print(knn.score(X_test, y_test))
print()

y_pred_ = knn.predict(X_test)

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred_))
print()

print("Classification report:")
print(classification_report(y_test, y_pred_))

The knn score suggest there are somewhat tight relationships with the data.  However, the "high" mortality classification prediction is not as high suggesting mortality from COVID is not that correlated to the gdp per capita or the population density

In [ ]:
#model complexity
train_accuracies = {}
test_accuracies = {}
neighbors = np.arange(1,26) #(1,26)


In [ ]:
# loop through neighbors array
for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors=neighbor)
    knn.fit(X_train, y_train)
    train_accuracies[neighbor]=knn.score(X_train, y_train)
    test_accuracies[neighbor]=knn.score(X_test,y_test)

In [ ]:
# plot training and test values
#plt.figure(figuresize=(8,6))
plt.title("KNN: varying number of neighbors")
plt.plot(neighbors, train_accuracies.values(), label="Training Accuracy")
plt.plot(neighbors, test_accuracies.values(), label="Testing Accuracy")
plt.legend()
plt.xlabel("Number of Neighbors")
plt.ylabel("Accuracy")

plt.show()


Tried with 5, 6, then 13 but this shows k of 3 is a good choice as this displays the highest testing accuracy and training score

<h4>Supervised learning with regression</h4>

In [ ]:
#training and testing the data
#from datacamp

#covid_data_ml
#covid_data_ml1
#covid_data_ml2 
#print(ml_data)

ml_data = covid_data_ml1


X = ml_data.drop("total_deaths_per_100k",axis=1).values #drop target value
y = ml_data["total_deaths_per_100k"].values #target observations


# predicting mortality using population density

#predict using pop_density (6)
X_pop_d = X[:,6] 
#print(y.shape, X_pop_d.shape) # check shape
# reshape
X_pop_d = X_pop_d.reshape(-1,1) 
#print(X_pop_d.shape) #check shape

#regression model
reg = LinearRegression()
reg.fit(X_pop_d,y)
predictions = reg.predict(X_pop_d)
print(predictions[:10])

#plot Total_deaths per 100k vs. population density with regression
plt.scatter(X_pop_d, y)
plt.plot(X_pop_d, predictions, color = "gray")
plt.ylabel("Mortality per 100k")
plt.xlabel("Population density")
plt.show()


Weak negative correlation. The higher the population density the less likey the mortality from COVID, this is unexpected. I would expect the line to be positive

In [ ]:
# predicting mortality using gdp

#predict using gdp_per_capita (7)
X_gdp_c = X[:,7] 

#print(ml_data)

# reshape
X_gdp_c = X_gdp_c.reshape(-1,1) 
#print(X_gdp_c.shape) #check shape

#regression model
reg = LinearRegression()
reg.fit(X_gdp_c,y)
predictions = reg.predict(X_gdp_c)
print(predictions[:10])

#plot Total_deaths per 100k vs. population density
plt.scatter(X_gdp_c, y)
plt.plot(X_gdp_c, predictions, color = "gray")
plt.ylabel("Mortality per 100k")
plt.xlabel("GDP per Capita")
plt.show()


Weak positive correlation. The higher the gdp per capita the less likey the mortality from COVID, this is somewhat expected, I would have expected the line to be steeper and negative.

In [ ]:
#Linear regression using all features

# need to drop mortality
covid_data_sns.drop(["date","iso_code","location","total_cases", "total_deaths","mortality"],axis=1)


ml_data_r = covid_data_sns.drop(["date","iso_code","location","total_cases", "total_deaths","mortality"],axis=1)

ml_data_r



X = ml_data_r.drop("total_deaths_per_100k",axis=1).values #drop target value
y = ml_data_r["total_deaths_per_100k"].values #target observations


#split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=21)
knn = KNeighborsClassifier(n_neighbors=5)

#fit the linear regression to the training data
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)


#predict on the test set
y_pred = reg_all.predict(X_test)

r_score = reg_all.score(X_test, y_test)

print("Predictions: {}, Actual Values: {}".format(y_pred[:4], y_test[:4]))
print("There are large gaps between the predictions and test data")
print("The model only explains about %5.2f"%(r_score*100)+"% of mortality level variance" )


<h6>Results summary</h6>

<pre style="font-family:arial;">
    Per the charts and analysis above, the results are not encoraging based on my inital hypothesis: that higher population density and lower GDP per capita for a country would have a negative impact on COVID mortality (higher deaths).  I beleive there may be some outliers, as seen in the scatter plot data, that should be reviewed further. This would potentially provide better results.
    
    I would have liked to had time to understand better the impact of the country specifc attributes on the data because my feeling is the scatter plot data does not appear to me to correlate to the result data.  Using these results it does not appear that we can reply strongly on GDP or population to forecast mortalitly.  I would be nice to continue this and test against other policies like masking and stay at home orders to see what an impact it had and if those policies impacted this disease.
    
    Overall, the data shows some correlations but fairly weak. the k score looked promising at .702 and the Classification report F1 score of 0.79/0.78 was ok performance. The confusion matrix results were ok (18 true positive and 15 for the false negative while 7 false positives compared to 48 true negative.  I think this may have been skewed by the fairly wide grouping I gave for "high" mortality vs "low". I should probably have left the "mortality" field as just "high" as it made understanding the confusion matirx more difficult, or confusing.
    
    Given the flatness of the regression line it would make sense to review some of the outlier data and rerun maybe with a wider set of data.
    
    I reviewed the hypertuning and I' not condifent it will imporve the performace of the report by much and given the lower accuracy I do not think over fitting is an issue.  For those reasons I chose not to use the hyper parameter tuning and focus on the machine learning module
</pre>

<h2 style="color:#00008b;">Insights</h2>
(Point out at least 5 insights in bullet points)

* GDP and population density do not appear to be strongly correlated to mortality due to COVID.  I would like to have used the country data better the machine learning would probably give better insights into the correlation
* The negative skew on the population density was quite surprising.  I expected the denser a population the greater the chance of COVID spearding
* I really expected there to be a tighter correlation between the data and need more time to review the data for items that could be corrected
* Some of the visuals can become overly complex and it takes time to simplify the visual into a basic form for interpretation
* The amount of information to learn about python and the amount of insights is daunting and takes patience and perseverence

<h2 style="color:#00008b;">References</h2>
HTML Code help: <a href="https://www.w3schools.com/html/html_links.asp" target="_blank">W3 Schools</a> 

Our World in Data (OWID): <a href="https://ourworldindata.org/coronavirus#explore-the-global-situation" target="_blank">https://ourworldindata.org/coronavirus#explore-the-global-situation</a> 

The World Bank GDP: <a href="https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?year_high_desc=false" target="_blank">https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?year_high_desc=false</a>

Python:
    <p>formatting numbers: https://pythonguides.com/python-format-number-with-commas/#:~:text=Python%20format%20number%20with%20commas%20Let%20us%20see,comma%20to%20every%20thousand%20places%20starting%20from%20left.</p>
    <p>formatting dates: https://stackabuse.com/how-to-format-dates-in-python/</p>
    

<h2 style="color:#00008b;">Other Techniques Demonstrated</h2>

Merge DataFrames COVID_data to GDP data

In [ ]:
covid__data_merge1 = covid_data[covid_data["date"].isin([last_date_n])]

#rename location to Country_Name
covid__data_merge1.rename(columns = {'location':'Country_Name'}, inplace = True)

print("\nAfter modifying column:\n", covid__data_merge1.columns)

In [ ]:
gdp_data.head()

In [ ]:
#GDP NaN values 
gdp_data.isna().sum()

In [ ]:
#deal with Nan values for GDP data
#fill NaN with an average
fill_value = pd.DataFrame({col: gdp_data.mean(axis=1) for col in gdp_data.columns})
gdp_data.fillna(fill_value, inplace=True)


gdp_data.head()

In [ ]:
#merge Covid to Gdp
df_covid_merge =pd.merge(covid__data_merge1,gdp_data, how="outer")

print(df_covid_merge.columns)
df_covid_merge.head()

In [ ]:
#regedit to find a strin of characters:

# I did not use this code in the filtering above becuase tehre were other easier ways to find the list of OWID iso codes

test = "how am i to find OWID_ or owid or 'OWID' any other set of strings"

headers=covid_data_raw["iso_code"].tolist()
unique_headers = unique(headers)


owid = re.findall("OWID",str(headers))
print(unique(owid))

if owid:
  print("I found a match")
else:
  print("No match")


